In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingRegressor
import doubleml as dml
import patsy

# Define random state
np.random.seed(123)
rs = 123

In [ ]:
# Import fullly preprocessed dataset
filepath = "../data_processing/processed_merged_data/full_dataset.csv"
df = pd.read_csv(filepath)

In [ ]:
# Log transform GVA_Real_Density and Tax_D_Real
df['GVA_Real_Density'] = np.log(df['GVA_Real_Density'])
df['Tax_D_Real'] = np.log(df['Tax_D_Real'])

In [ ]:
# Define the cutoff values based on the propensity score analysis
tax_cutoff = 7.2 # Justify with generalised propensity score
gva_cutoff = 11 # Justify with scatter plot

# Find LANMs that are outside of range consistently
outliers = df.groupby('LANM').filter(
    lambda x: x['Tax_D_Real'].mean() < tax_cutoff and 
    x['GVA_Real_Density'].mean() > gva_cutoff
)

outliers = outliers['LANM'].unique()
print(f"List of Outlier LAs: {outliers}")

# Remove outliers from the dataset
df = df[~df['LANM'].isin(outliers)]

# Calculate total missing rows
num_years = df['Year'].nunique()
dropped_rows = len(outliers) * num_years
print(f"Number of rows dropped: {dropped_rows}")
print(f"Rows of data remaining: {df.shape[0]}")

In [ ]:
# Remove alternate outcome "GVA_Real"
ml_df = df.drop(columns=['GVA_Real'])

# Remove all cols with GDHI in the name (mediators)
ml_df = ml_df.loc[:, ~ml_df.columns.str.contains('GDHI')]

# Remove non time-varying variables
ml_df = ml_df.drop(columns=['Area', 'Region', 'Authority_Type'])

# Remove id name columns
ml_df = ml_df.drop(columns=['LANM'])

# Duplicate year column
ml_df['Year_Group'] = ml_df['Year'].astype('str')

# Duplicate LACD column
ml_df['LACD_Group'] = ml_df['LACD'].astype('str')

# Create one hot encoding for fixed effects
ml_df = pd.get_dummies(ml_df, columns=['LACD'])

# Show columns and number of columns
print(ml_df.columns)
print(len(ml_df.columns))

In [ ]:
# Define Nuisance Parameter Learners
ml_l = GradientBoostingRegressor(random_state=rs)
ml_m = GradientBoostingRegressor(random_state=rs)

# Define two-dimensional clustering
clusters = ['LACD_Group', 'Year_Group']

# Create DoubleML cluster data object
obj_dml_data = dml.DoubleMLClusterData(ml_df,
                                       'GVA_Real_Density',
                                       'Tax_D_Real',
                                       clusters)

# Create DoubleML Partially Linear Regression object
dml_plr_obj = dml.DoubleMLPLR(obj_dml_data, ml_l, ml_m)

# Define parameter grids for tuning
par_grids = {'ml_l': {'n_estimators': [300, 400, 600, 800],
                      'learning_rate': [0.01, 0.05, 0.1, 0.2],
                      'max_features': [0.2, 0.5, 0.8],
                      'max_depth': [3, 5, 10, 30],
                      'min_samples_split': [2, 4, 8],
                      'min_samples_leaf': [1, 2, 5]},
             'ml_m': {'n_estimators': [400, 600, 800, 1000],
                      'learning_rate': [0.01, 0.05, 0.1, 0.2],
                      'max_features': [0.2, 0.5, 0.8],
                      'max_depth': [3, 5, 10, 30],
                      'min_samples_split': [2, 4, 8],
                      'min_samples_leaf': [1, 2, 5]}}

# Tune hyperparameters
dml_plr_obj.tune(par_grids, search_mode='randomized_search',
                 n_iter_randomized_search=50)

# Print optimal hyperparameters
print(dml_plr_obj.params)

In [ ]:
# Fit DML model
print(dml_plr_obj.fit())